In [3]:
# Import required libraries

from datasets import load_dataset
from dotenv import load_dotenv
from huggingface_hub import HfApi, HfFolder
import markdown
import os
import pandas as pd
import requests
import subprocess
import transformers

In [4]:
# Log in to Hugging Face

load_dotenv()

hf_token = os.getenv('HF_TOKEN')
HfFolder.save_token(hf_token)


result = subprocess.run(["huggingface-cli", "login", "--token", hf_token], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

if result.returncode == 0:
    print("Successfully logged in to Hugging Face.")
else:
    print("Failed to log in to Hugging Face.")
    print(result.stderr.decode())

Successfully logged in to Hugging Face.


In [5]:
# Collect and print MPEP datasets

api = HfApi()

def search_datasets_by_label(label):
    datasets = api.list_datasets(search=label)
    return datasets

mpep_datasets = search_datasets_by_label('MPEP')

# Extract and print the first entry's value parameter from the target JSON object for each dataset
for dataset in mpep_datasets:
    ds = load_dataset(dataset.id)
    target_entries = ds['train']['target'][:1]
    print(f"Dataset: {dataset.id}")
    if target_entries:
        first_target = target_entries[0]
        if isinstance(first_target, list) and first_target:
            value = first_target[0].get('value', 'No value found')
            print(value)
        elif isinstance(first_target, dict):
            value = first_target.get('value', 'No value found')
            print(value)
        else:
            print("No target value found")
    else:
        print("No target entries found")
    print("\n" + "-"*40 + "\n")

Dataset: DIBT/MPEP_DUTCH
Als een AI-enthousiasteling, houd je ervan om programma's te maken die de menselijke taal begrijpen. Je nieuwste project is een programma dat woorden kan herkennen en vervangen door hun antoniemen in een stuk tekst.
Om de effectiviteit van je programma aan te tonen, besluit je het te testen op een nieuwsartikel over een recent politiek evenement. Om het uitdagender te maken, wil je ook dat je programma onderscheid maakt tussen homoniemen, en daarnaast contextuele aanwijzingen gebruikt woorden correct te vervangen.
Hier is een stapsgewijze uitleg van hoe je programma werkt:
1. Het programma leest de inputtekst en identificeert alle woorden die antoniemen hebben.
2. Voor elk van deze woorden identificeert het programma de context waarin het verschijnt om het juiste antoniem te bepalen.
3. Het programma vervangt vervolgens het originele woord door zijn antoniem in de tekst.
4. Als het originele woord meerdere betekenissen heeft, gebruikt het programma de context o

Generating train split: 100%|██████████| 26/26 [00:00<00:00, 1210.26 examples/s]


Dataset: DIBT/MPEP_MALAGASY
Mila CV aho ho an'ity asa ity
Junior PHP injeniera (f / m / d)
Momba ny asa
Anstellungsdetails

Fifanekena maharitra, fotoana feno na tapak'andro, Cologne / Düsseldorf / Darmstadt / Remote (any Alemaina)

Info

Iza moa isika

Miasa Kaufland.de izahay: mpivarotra an'arivony sy vokatra an-tapitrisany no mahatonga antsika ho iray amin'ireo tsena an-tserasera mitombo haingana indrindra. Ny asanay dia miavaka amin'ny kolontsain'ny orinasa mavitrika, miaraka amin'ny toe-tsaina manomboka sy ny herin'ny vondrona orinasa lehibe. Manambatra ny fahalalana sy ny traikefa an-taonany maro amin'ny e-varotra miaraka amin'ny ambaratongam-pahefana sy ekipa tena manosika izahay. Na avy amin'ny intern: Raisinay ho zava-dehibe ny hevitra rehetra, satria te-hiara-hiasa amin'ny famolavolana ny hoavin'ny e-varotra izahay!






----------------------------------------

Dataset: DIBT/MPEP_GERMAN
Kannst du zwei Beispiele für Metaphern im Excel-Tabellenformat liefern?

Hier ist eine E

In [6]:
def query_hf_inference(model, prompt, token):
    api_url = f"https://api-inference.huggingface.co/models/{model}"
    headers = {"Authorization": f"Bearer {token}"}
    try:
        response = requests.post(api_url, headers=headers, json={"inputs": prompt}, timeout=30)
        if response.status_code == 200:
            try:
                result = response.json()
                if isinstance(result, list) and len(result) > 0:
                    return result[0].get('generated_text', 'No generated text found')
                else:
                    return "Unexpected response format"
            except ValueError:
                return "Invalid JSON response"
        else:
            return f"Error {response.status_code}: {response.text}"
    except requests.exceptions.Timeout:
        return "Request timed out"


# List of target models
models = [
    "google/gemma-2-27b-it",
    "meta-llama/Meta-Llama-3-70B-Instruct",
    "Qwen/Qwen2-72B-Instruct",
    "mistralai/Mixtral-8x22B-Instruct-v0.1",
]

# Number of prompts to process
num_prompts = 5

def search_datasets_by_label(label):
    # Search datasets with the specified label
    datasets = api.list_datasets(search=label)
    return datasets

# Search for datasets with the "MPEP" label
mpep_datasets = search_datasets_by_label('MPEP')

# Directory to save responses
os.makedirs('./responses', exist_ok=True)

# Process each dataset
for dataset in mpep_datasets:
    # Load the dataset
    ds = load_dataset(dataset.id)
    # Extract the first 'num_prompts' entries from the target JSON object
    target_entries = ds['train']['target'][:num_prompts]
    
    # Get the language from the dataset id (assuming the format is consistent)
    language = dataset.id.split('_')[-1].lower()
    
    # Open a file to write the responses
    with open(f'./responses/{language}.md', 'w') as file:
        # Capture responses for each model
        for model in models:
            file.write(f"# Model: {model}\n\n")
            for target_entry in target_entries:
                if isinstance(target_entry, list) and target_entry:
                    prompt = target_entry[0].get('value', 'No value found')
                elif isinstance(target_entry, dict):
                    prompt = target_entry.get('value', 'No value found')
                else:
                    prompt = "No prompt found"
                
                response_text = query_hf_inference(model, prompt, os.getenv('HF_TOKEN'))
                file.write(f"## Prompt:\n{prompt}\n\n")
                file.write(f"## Response:\n{response_text}\n\n")
                file.write("\n" + "-"*40 + "\n\n")

print("Script completed successfully.")

Script completed successfully.
